In [1]:
# v1: out_channels=512
# v2: out_channels=128 inceptionc -> 128

In [2]:
import numpy as np
import pandas as pd
import torch
from torch import Tensor
import torch.nn as nn
from typing import Callable, Any, Optional, Tuple, List
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR, CosineAnnealingWarmRestarts, ReduceLROnPlateau
from sklearn.cluster import KMeans
from sklearn.model_selection import StratifiedKFold
import random
import os
from logging import getLogger, INFO, FileHandler, Formatter, StreamHandler
import copy
import time
from sklearn.metrics import mean_absolute_error

In [3]:
# InceptionA and InceptionC
# Copied from https://github.com/pytorch/vision/blob/master/torchvision/models/inception.py
# Copied from https://amaarora.github.io/2020/07/24/SeNet.html

class BasicConv1d(nn.Module):

    def __init__(
        self,
        in_channels: int,
        out_channels: int,
        **kwargs: Any
    ) -> None:
        super(BasicConv1d, self).__init__()
        self.conv = nn.Conv1d(in_channels, out_channels, bias=False, **kwargs)
        self.bn = nn.BatchNorm1d(out_channels, eps=0.001)

    def forward(self, x: Tensor) -> Tensor:
        x = self.conv(x)
        x = self.bn(x)
        return F.leaky_relu(x, inplace=True)

    
class InceptionA(nn.Module):

    def __init__(
        self,
        in_channels: int,
        pool_features: int,
        conv_block: Optional[Callable[..., nn.Module]] = None
    ) -> None:
        super(InceptionA, self).__init__()
        if conv_block is None:
            conv_block = BasicConv1d
        self.branch1x1 = conv_block(in_channels, 64, kernel_size=1)

        self.branch5x5_1 = conv_block(in_channels, 48, kernel_size=1)
        self.branch5x5_2 = conv_block(48, 64, kernel_size=5, padding=2)

        self.branch3x3dbl_1 = conv_block(in_channels, 64, kernel_size=1)
        self.branch3x3dbl_2 = conv_block(64, 96, kernel_size=3, padding=1)
        self.branch3x3dbl_3 = conv_block(96, 96, kernel_size=3, padding=1)

        self.branch_pool = conv_block(in_channels, pool_features, kernel_size=1)

    def _forward(self, x: Tensor) -> List[Tensor]:
        branch1x1 = self.branch1x1(x)

        branch5x5 = self.branch5x5_1(x)
        branch5x5 = self.branch5x5_2(branch5x5)

        branch3x3dbl = self.branch3x3dbl_1(x)
        branch3x3dbl = self.branch3x3dbl_2(branch3x3dbl)
        branch3x3dbl = self.branch3x3dbl_3(branch3x3dbl)

        branch_pool = F.avg_pool2d(x, kernel_size=3, stride=1, padding=1)
        branch_pool = self.branch_pool(branch_pool)

        outputs = [branch1x1, branch5x5, branch3x3dbl, branch_pool]
        return outputs

    def forward(self, x: Tensor) -> Tensor:
        outputs = self._forward(x)
        return torch.cat(outputs, 1)


class InceptionC(nn.Module):

    def __init__(
        self,
        in_channels: int,
        channels_7x7: int,
        conv_block: Optional[Callable[..., nn.Module]] = None
    ) -> None:
        super(InceptionC, self).__init__()
        if conv_block is None:
            conv_block = BasicConv1d
        self.branch1x1 = conv_block(in_channels, 128, kernel_size=1)

        c7 = channels_7x7
        self.branch7x7_1 = conv_block(in_channels, c7, kernel_size=1)
        self.branch7x7_2 = conv_block(c7, c7, kernel_size=1, padding=0)
        self.branch7x7_3 = conv_block(c7, 128, kernel_size=7, padding=3)

        self.branch7x7dbl_1 = conv_block(in_channels, c7, kernel_size=1)
        self.branch7x7dbl_2 = conv_block(c7, c7, kernel_size=7, padding=3,)
        self.branch7x7dbl_3 = conv_block(c7, c7, kernel_size=1, padding=0,)
        self.branch7x7dbl_4 = conv_block(c7, c7, kernel_size=7, padding=3,)
        self.branch7x7dbl_5 = conv_block(c7, 128, kernel_size=1, padding=0,)

        self.branch_pool = conv_block(in_channels, 128, kernel_size=1)

    def _forward(self, x: Tensor) -> List[Tensor]:
        branch1x1 = self.branch1x1(x)

        branch7x7 = self.branch7x7_1(x)
        branch7x7 = self.branch7x7_2(branch7x7)
        branch7x7 = self.branch7x7_3(branch7x7)

        branch7x7dbl = self.branch7x7dbl_1(x)
        branch7x7dbl = self.branch7x7dbl_2(branch7x7dbl)
        branch7x7dbl = self.branch7x7dbl_3(branch7x7dbl)
        branch7x7dbl = self.branch7x7dbl_4(branch7x7dbl)
        branch7x7dbl = self.branch7x7dbl_5(branch7x7dbl)

        branch_pool = F.avg_pool1d(x, kernel_size=3, stride=1, padding=1)
        branch_pool = self.branch_pool(branch_pool)

        outputs = [branch1x1, branch7x7, branch7x7dbl, branch_pool]
        return outputs

    def forward(self, x: Tensor) -> Tensor:
        outputs = self._forward(x)
        return torch.cat(outputs, 1)
    

class SE_Block(nn.Module):
    "credits: https://github.com/moskomule/senet.pytorch/blob/master/senet/se_module.py#L4"
    def __init__(self, c, r=16):
        super().__init__()
        self.squeeze = nn.AdaptiveAvgPool1d(1)
        self.excitation = nn.Sequential(
            nn.Linear(c, c // r, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(c // r, c, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        bs, c, _ = x.shape
        y = self.squeeze(x).view(bs, c)
        y = self.excitation(y).view(bs, c, 1)
        return x * y.expand_as(x)

In [4]:
class CFG:
    lr = 0.005
    min_lr = 1e-6
    weight_decay = 0.001
    epochs = 1200
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    seed = 42
    NFOLD = 5
    batch_size = 128
    DATA_ROOT = r'../input/2nd-solution-reproduce-feature-part/train'
    OUTPUT_DIR = r'./'
    scheduler='CosineAnnealingWarmRestarts'#
    target_col = 'time_to_eruption_normalize'
    model_name = '2nd_model'
    # for warm start
    T_0 = 10
    # for normal cosine
    T_max = 10
    num_workers = 1
    
    EARLY_STOP = True
    early_stop = 20
    
    gradient_accumulation_steps=1
    max_grad_norm = 1000
    print_freq = 20

In [5]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG.seed)

In [6]:
class MyModel(nn.Module):
    def __init__(self, in_channels=340, out_channels=128, pool_features=64):
        super(MyModel, self).__init__()
        self.bn1 = nn.BatchNorm1d(in_channels)
        self.stem1 = nn.Sequential(nn.Conv1d(in_channels, out_channels, 3, padding=1), 
                                   nn.BatchNorm1d(out_channels), 
                                   # N,C,L => N,C,Lout
                                   nn.MaxPool1d(2,2))
        self.stem2 = nn.Sequential(nn.Conv1d(out_channels, out_channels, 3, padding=1), 
                                   nn.BatchNorm1d(out_channels), 
                                   nn.MaxPool1d(2,2))
        self.inceptiona = InceptionA(in_channels=out_channels, pool_features=pool_features)
        self.inceptionc = InceptionC(in_channels=out_channels, channels_7x7=out_channels)
        
        self.SE_Block = SE_Block(928)
        # (N, C, L)
        self.Bi_LSTM = nn.LSTM(input_size=928, 
                               hidden_size=128,
                               batch_first=True, 
                               bidirectional=True)
        self.adaptive_pooling = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Linear(29, 1)
        
    def forward(self, x):
        x = self.bn1(x)
        x = F.leaky_relu(x)
        x = self.stem1(x)
        x = F.leaky_relu(x)
        x = self.stem2(x)
        x = F.leaky_relu(x)
        x1 = self.inceptiona(x)
        x2 = self.inceptionc(x)
        x = torch.cat([x, x1, x2], dim=1)
        x = F.leaky_relu(x)
        x = self.SE_Block(x)
        x = F.leaky_relu(x)
        x = self.Bi_LSTM(x.permute(0,2,1))
        x = self.adaptive_pooling(x[0])
        x = F.relu(x)
        x = self.fc(x.squeeze(-1))
        
        return x

In [7]:
class INGVDataset(Dataset):
    def __init__(self, df, data_root, transforms=None, output_label=True):
        super().__init__()
        self.df = df
        self.data_root = data_root
        self.transforms = transforms
        self.output_label = output_label
    
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, index: int):
        
        # get labels
        if self.output_label:
            #target = self.df.iloc[index]['label']
            target = [self.df.loc[index, 'time_to_eruption_normalize'].astype('float32')]
          
        path = os.path.join(self.data_root, str(self.df.loc[index, 'segment_id'])+'.npz')
        
        # (10, 256, 256)
        img = np.load(path)['arr_0'].astype('float32')
            
        # do label smoothing
        if self.output_label == True:
            return torch.tensor(img), torch.tensor(target)
        else:
            return torch.tensor(img)

In [8]:
df = pd.read_csv(r'../input/predict-volcanic-eruptions-ingv-oe/train.csv')
km = KMeans(n_clusters=5)
df['clusters'] = km.fit_predict(df['time_to_eruption'].values.reshape(-1,1))

# take one fold as example
skf = StratifiedKFold(n_splits=CFG.NFOLD)
df['fold'] = 0
time2eruption_mean = df['time_to_eruption'].mean(axis=0)
for trn_idx, val_idx in skf.split(df['clusters'].values.reshape(-1,1), df['clusters'].values.reshape(-1,1)):
    df.loc[trn_idx, 'fold'] = 1
    break

train_df = df.loc[df['fold'] == 1].reset_index(drop=True)
valid_df = df.loc[df['fold'] != 1].reset_index(drop=True)
train_df['time_to_eruption_normalize'] = train_df['time_to_eruption'] / time2eruption_mean
valid_df['time_to_eruption_normalize'] = valid_df['time_to_eruption'] / time2eruption_mean

In [9]:
# 记录训练过程
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        
        self.true_val = 0
        self.true_avg = 0
        self.true_sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count
        
    def true_update(self, val, n=1):
        self.true_val = val * time2eruption_mean
        self.true_sum += self.true_val * n
        self.count += n
        self.true_avg = self.true_sum / self.count
        
def asMinutes(s):
    m = s // 60
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (remain %s)' % (asMinutes(s), asMinutes(rs))

class LogCoshLoss(torch.nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, y_t, y_prime_t):
        ey_t = y_t - y_prime_t
        return torch.mean(torch.log(torch.cosh(ey_t + 1e-12)))

In [10]:
def train_fn(train_loader, model, epoch, criterion, optimizer, scheduler, device):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    scores = AverageMeter()
    
    # switch to train mode
    model.train()
    
    start = end = time.time()
    global_step = 0
    for step, (images, labels) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)
        # torch.Size([32, 10, 256, 256])
        # print(images.shape)
        # torch.Size([32, 1])
        # print(labels.shape)
        images = images.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)
        y_preds = model(images)
        # torch.Size([32, 1])
        # print(y_preds.shape)
        loss = criterion(labels, y_preds)
        # record loss
        losses.update(loss.item(), batch_size)
        losses.true_update(loss.item(), batch_size)
        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
        loss.backward()
        
        grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), CFG.max_grad_norm)
        if (step + 1) % CFG.gradient_accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()
            global_step += 1
            
        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        
        if step % CFG.print_freq == 0 or step == (len(train_loader)-1):
            print('Epoch: [{0}][{1}/{2}] '
                  'Data {data_time.val:.3f} ({data_time.avg:.3f}) '
                  'Elapsed {remain:s} '
                  'Loss: {loss.val:.4f}({loss.avg:.4f}) {loss.true_val:.4f}({loss.true_avg:.4f})'
                  'Grad: {grad_norm:.4f}  '
                  #'LR: {lr:.6f}  '
                  .format(
                   epoch+1, step, len(train_loader), batch_time=batch_time,
                   data_time=data_time, loss=losses,
                   remain=timeSince(start, float(step+1)/len(train_loader)),
                   grad_norm=grad_norm,
                   #lr=scheduler.get_lr()[0],
                   ))
    return losses.avg


def valid_fn(valid_loader, model, criterion, device):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    scores = AverageMeter()
    # switch to evaluation mode
    model.eval()
    preds = []
    start = end = time.time()
    for step, (images, labels) in enumerate(valid_loader):
        # measure data loading time
        data_time.update(time.time() - end)
        images = images.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)
        # compute loss
        with torch.no_grad():
            y_preds = model(images)
        loss = criterion(labels, y_preds)
        losses.update(loss.item(), batch_size)
        losses.true_update(loss.item(), batch_size)
        # record accuracy
        # preds.append(y_preds.softmax(1).to('cpu').numpy())
        preds.append(y_preds.to('cpu').numpy())
        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        if step % CFG.print_freq == 0 or step == (len(valid_loader)-1):
            print('EVAL: [{0}/{1}] '
                  'Data {data_time.val:.3f} ({data_time.avg:.3f}) '
                  'Elapsed {remain:s} '
                  'Loss: {loss.val:.4f}({loss.avg:.4f}) {loss.true_val:.4f}({loss.true_avg:.4f})'
                  .format(
                   step, len(valid_loader), batch_time=batch_time,
                   data_time=data_time, loss=losses,
                   remain=timeSince(start, float(step+1)/len(valid_loader)),
                   ))
    predictions = np.concatenate(preds)
    
    return losses.avg, predictions

def init_logger(log_file=CFG.OUTPUT_DIR+'train.log'):
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=log_file)
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

def get_score(y_true, y_pred):
    return mean_absolute_error(y_true, y_pred)

In [11]:
# ====================================================
# Train loop
# ====================================================

LOGGER = init_logger()

def train_loop(train_df, valid_df):

    LOGGER.info(f"========== start training ==========")
    
    # 存储结果
    res_valid_df = copy.deepcopy(valid_df)
    # ====================================================
    # loader
    # ====================================================

    train_dataset = INGVDataset(train_df, CFG.DATA_ROOT)
    valid_dataset = INGVDataset(valid_df, CFG.DATA_ROOT)

    train_loader = DataLoader(train_dataset, batch_size=CFG.batch_size, num_workers=CFG.num_workers, shuffle=True)
    valid_loader = DataLoader(valid_dataset, batch_size=CFG.batch_size, num_workers=CFG.num_workers, shuffle=False)
    
    # model
    model = MyModel()
    model.to(CFG.device)
    
    # optimizer
    optimizer = torch.optim.AdamW(model.parameters(), lr=CFG.lr, weight_decay=CFG.weight_decay)
    
    # ====================================================
    # scheduler 
    # ====================================================
    if CFG.scheduler=='CosineAnnealingLR':
        scheduler = CosineAnnealingLR(optimizer, T_max=CFG.T_max, eta_min=CFG.min_lr, last_epoch=-1)
    elif CFG.scheduler=='CosineAnnealingWarmRestarts':
        scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=CFG.T_0, T_mult=2, eta_min=CFG.min_lr, last_epoch=-1)
    elif CFG.scheduler=='ReduceLROnPlateau':
        scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=10, verbose=True)
    else:
        raise Exception('scheduler not set')


    # ====================================================
    # loop
    # ====================================================
    criterion = nn.L1Loss()#LogCoshLoss()

    best_score = float('inf')
    best_loss = np.inf
    early_round = 0
    
    for epoch in range(CFG.epochs):
        
        start_time = time.time()
        
        # =================================================
        # train
        avg_loss = train_fn(train_loader, model, epoch, criterion, optimizer, scheduler, CFG.device)

        # eval
        avg_val_loss, preds = valid_fn(valid_loader, model, criterion, CFG.device)
        valid_labels = valid_df[CFG.target_col].values
        
        if isinstance(scheduler, torch.optim.lr_scheduler.CosineAnnealingLR):
            scheduler.step()
        elif isinstance(scheduler, torch.optim.lr_scheduler.CosineAnnealingWarmRestarts):
            scheduler.step()
        elif isinstance(scheduler, torch.optim.lr_scheduler.ReduceLROnPlateau):
            scheduler.step(avg_val_loss)

        # scoring
        score = get_score(valid_labels, preds)

        elapsed = time.time() - start_time

        LOGGER.info(f'Epoch {epoch+1} - avg_train_loss: {avg_loss:.4f}  avg_val_loss: {avg_val_loss:.4f}  time: {elapsed:.0f}s')
        LOGGER.info(f'Epoch {epoch+1} - MAE: {score}')
        
#         save_checkpoint(model, optimizer, scheduler, epoch, fold)
#         print('===================== checkpoint saved =======================')
        
        if score < best_score:
            best_score = score
            LOGGER.info(f'Epoch {epoch+1} - Save Best Score: {best_score:.4f} Model')
            torch.save({'model': model.state_dict(), 
                        'preds': preds},
                        CFG.OUTPUT_DIR+f'{CFG.model_name}_best.pth')
            early_round = 0
            
        else:
            if CFG.EARLY_STOP:
                early_round += 1
                if early_round == CFG.early_stop:
                    LOGGER.info('===================== Early Stop =====================')
                    break
    
    check_point = torch.load(CFG.OUTPUT_DIR+f'{CFG.model_name}_best.pth')
    res_valid_df['preds'] = check_point['preds']

    return res_valid_df

In [12]:
res_valid_df = train_loop(train_df, valid_df)

========== start training ==========


Epoch: [1][0/28] Data 3.521 (3.521) Elapsed 0m 4s (remain 1m 54s) Loss: 0.9866(0.9866) 22542517.2466(11271258.6233)Grad: 0.9689  
Epoch: [1][20/28] Data 1.860 (2.004) Elapsed 0m 43s (remain 0m 14s) Loss: 0.3154(0.2677) 7206442.4918(5970661.8479)Grad: 1.1613  
Epoch: [1][27/28] Data 1.298 (1.951) Elapsed 0m 57s (remain 0m 0s) Loss: 0.3016(0.2384) 6891297.2442(5379026.7094)Grad: 1.2545  
EVAL: [0/7] Data 2.209 (2.209) Elapsed 0m 2s (remain 0m 13s) Loss: 0.2596(0.2596) 5931135.9472(2965567.9736)


Epoch 1 - avg_train_loss: 0.2384  avg_val_loss: 0.1498  time: 71s
Epoch 1 - MAE: 0.27947450550788366
Epoch 1 - Save Best Score: 0.2795 Model


EVAL: [6/7] Data 1.818 (1.933) Elapsed 0m 13s (remain 0m 0s) Loss: 0.3265(0.1498) 7460053.6485(3192843.4554)
Epoch: [2][0/28] Data 0.289 (0.289) Elapsed 0m 0s (remain 0m 9s) Loss: 0.2070(0.2070) 4730457.8308(2365228.9154)Grad: 0.1206  
Epoch: [2][20/28] Data 0.184 (0.194) Elapsed 0m 5s (remain 0m 1s) Loss: 0.1707(0.1091) 3899817.4022(2433858.2671)Grad: 0.3485  
Epoch: [2][27/28] Data 0.109 (0.188) Elapsed 0m 6s (remain 0m 0s) Loss: 0.1730(0.1046) 3952217.2232(2361329.4962)Grad: 0.2216  
EVAL: [0/7] Data 0.274 (0.274) Elapsed 0m 0s (remain 0m 1s) Loss: 0.1950(0.1950) 4454763.1092(2227381.5546)


Epoch 2 - avg_train_loss: 0.1046  avg_val_loss: 0.1061  time: 9s
Epoch 2 - MAE: 0.19791761063278251
Epoch 2 - Save Best Score: 0.1979 Model


...

EVAL: [6/7] Data 0.203 (0.226) Elapsed 0m 1s (remain 0m 0s) Loss: 0.0310(0.0175) 708244.7707(373206.7697)
Epoch: [87][0/28] Data 0.290 (0.290) Elapsed 0m 0s (remain 0m 9s) Loss: 0.0223(0.0223) 508905.5380(254452.7690)Grad: 0.7890  
Epoch: [87][20/28] Data 0.186 (0.193) Elapsed 0m 5s (remain 0m 1s) Loss: 0.0223(0.0115) 508940.0111(256480.4166)Grad: 0.6535  
Epoch: [87][27/28] Data 0.118 (0.192) Elapsed 0m 7s (remain 0m 0s) Loss: 0.0210(0.0113) 480505.0509(253885.2026)Grad: 0.7510  
EVAL: [0/7] Data 0.286 (0.286) Elapsed 0m 0s (remain 0m 1s) Loss: 0.0324(0.0324) 740286.9810(370143.4905)


Epoch 87 - avg_train_loss: 0.0113  avg_val_loss: 0.0172  time: 9s
Epoch 87 - MAE: 0.0320455844603677


EVAL: [6/7] Data 0.205 (0.246) Elapsed 0m 1s (remain 0m 0s) Loss: 0.0326(0.0172) 744230.1099(366103.2863)
Epoch: [88][0/28] Data 0.286 (0.286) Elapsed 0m 0s (remain 0m 10s) Loss: 0.0167(0.0167) 381873.5436(190936.7718)Grad: 0.9528  
Epoch: [88][20/28] Data 0.193 (0.198) Elapsed 0m 5s (remain 0m 1s) Loss: 0.0202(0.0100) 461594.1621(223628.2441)Grad: 1.0260  
Epoch: [88][27/28] Data 0.116 (0.194) Elapsed 0m 7s (remain 0m 0s) Loss: 0.0194(0.0101) 444362.4972(229035.5479)Grad: 0.8291  
EVAL: [0/7] Data 0.297 (0.297) Elapsed 0m 0s (remain 0m 1s) Loss: 0.0292(0.0292) 668092.2273(334046.1136)


Epoch 88 - avg_train_loss: 0.0101  avg_val_loss: 0.0166  time: 9s
Epoch 88 - MAE: 0.031065129202215287
===================== Early Stop =====================


EVAL: [6/7] Data 0.210 (0.236) Elapsed 0m 1s (remain 0m 0s) Loss: 0.0339(0.0166) 775638.1851(354902.1087)


In [13]:
res_valid_df.to_pickle('res_valid_df.pkl')